In [66]:
import pandas as pd
import matplotlib.pyplot as plt
from ast import literal_eval
import numpy as np
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from time import time

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.preprocessing import normalize
from scipy.sparse import csr_matrix, vstack
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier

import transformers as ppb # pytorch-pretrained-bert
import torch

import pipeline as pp
import models as ml

import importlib
import math

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\madsv\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [200]:
# The number of rows to train the model
BATCH_SIZE = 10000

In [108]:
pp.apply_pipeline(
    "../datasets/big/dataset.csv", 
    [(pp.Binary_labels(), 'type', 'type_binary')], 
    new_file="../datasets/big/dataset_bin.csv", 
    progress_bar=True
)

100%|██████████| 73069/73069 [00:00<00:00, 775856.99it/s]


finish time: 807.0935735702515


In [112]:
importlib.reload(pp)
from_file = "../datasets/big/dataset_bin.csv"

pp.get_dataframe_with_distribution(from_file, BATCH_SIZE, [0.8,0.1,0.1], [False, False, False], 
                                   out_file="../datasets/big/dataset_unbalanced.csv", get_frame=False)
pp.get_dataframe_with_distribution(from_file, BATCH_SIZE, [0.8,0.1,0.1], [True, True, False], 
                                   out_file="../datasets/big/dataset_balanced_types.csv", get_frame=False)
pp.get_dataframe_with_distribution(from_file, BATCH_SIZE, [0.8,0.1,0.1], [True, True, False],
                                   out_file="../datasets/big/dataset_balanced_bin.csv", get_frame=False, classes=[True,False], type_col="type_binary")

running


100%|██████████| 200000/200000 [00:00<00:00, 925826.46it/s]


entries read: 200000
running


100%|██████████| 200000/200000 [00:00<00:00, 672157.10it/s]


entries read: 200000
running


100%|██████████| 200000/200000 [00:00<00:00, 879091.55it/s]


entries read: 200000


In [135]:
def get_distribution(data, is_percentage=True, col = "type"):
    for i, label in enumerate(pp.labels):
        if is_percentage:
            percent = len(data[data[col] == label]) / (data.shape[0])
        else:
            percent = len(data[data[col] == label])
        print(f"{label}: {percent}", end="")
        print(", ", end="") if i != len(pp.labels) - 1 else _

for file in ["../datasets/big/dataset_unbalanced.csv", "../datasets/big/dataset_balanced_types.csv", "../datasets/big/dataset_balanced_bin.csv"]:
    data = pd.read_csv(file)
    print(f"File: {file} ----------------------------------")
    # find distribution of labels
    for i, set_name in enumerate(["train", "val", "test"]):
        set = data[data["set"] == i]
        print(f"Distribution of {set_name} with size {set.shape[0]}:")
        get_distribution(set)
        print(f"\nTrue: {len(set[set['type_binary'] == True])}, Fake: {len(set[set['type_binary'] == False])}")

File: ../datasets/big/dataset_unbalanced.csv ----------------------------------
Distribution of train with size 8000:
fake: 0.121625, conspiracy: 0.123125, junksci: 0.0155, hate: 0.01, unreliable: 0.04575, bias: 0.15475, satire: 0.016125, reliable: 0.259125, clickbait: 0.032, political: 0.222
True: 4105, Fake: 3895
Distribution of val with size 1000:
fake: 0.129, conspiracy: 0.124, junksci: 0.012, hate: 0.014, unreliable: 0.045, bias: 0.147, satire: 0.022, reliable: 0.262, clickbait: 0.021, political: 0.224
True: 507, Fake: 493
Distribution of test with size 1000:
fake: 0.106, conspiracy: 0.125, junksci: 0.017, hate: 0.008, unreliable: 0.05, bias: 0.172, satire: 0.014, reliable: 0.255, clickbait: 0.032, political: 0.221
True: 508, Fake: 492
File: ../datasets/big/dataset_balanced_types.csv ----------------------------------
Distribution of train with size 8000:
fake: 0.1, conspiracy: 0.1, junksci: 0.1, hate: 0.1, unreliable: 0.1, bias: 0.1, satire: 0.1, reliable: 0.1, clickbait: 0.1, po

In [ ]:
importlib.reload(pp)

def Clean_data(file, new_file):
    stopwords_lst = stopwords.words('english')
    pp.apply_pipeline(file, [
            # Clean content
            (pp.Clean_data(), 'content'),
            (pp.Tokenizer(), "content"),
            (pp.Remove_stopwords(stopwords_lst), "content"),
            (pp.Stem(), "content"),
            (pp.Combine_Content(), "content", "content_combined"),
            # Clean authors
            (pp.Clean_author(), "authors"),
            # Clean title
            (pp.Clean_data(), 'title'),
            (pp.Tokenizer(), "title"),
            (pp.Remove_stopwords(stopwords_lst), "title"),
            (pp.Stem(), "title"),
            (pp.Combine_Content(), "title"),
            # Clean domain
            (pp.Clean_domain(), 'domain'),
            # Combine columns (used as features)
            (pp.Join_str_columns(["content_combined", "authors"]), None, "content_authors"),
            (pp.Join_str_columns(["content_combined", "title"]), None, "content_title"),
            (pp.Join_str_columns(["content_combined", "domain"]), None, "content_domain"),
            (pp.Join_str_columns(["content_combined", "domain", "authors", "title"]), None, "content_domain_authors_title")
        ],
        new_file=new_file,
        progress_bar=True,
        nrows=BATCH_SIZE
    )

Clean_data("../datasets/big/dataset_unbalanced.csv", "../datasets/big/dataset_unbalanced_cleaned.csv")

In [181]:
Clean_data("../datasets/big/dataset_balanced_types.csv", "../datasets/big/dataset_balanced_types_cleaned.csv")
Clean_data("../datasets/big/dataset_balanced_bin.csv", "../datasets/big/dataset_balanced_bin_cleaned.csv")

100%|██████████| 10000/10000 [00:00<00:00, 27057.74it/s]


finish time: 98.6530590057373


100%|██████████| 10000/10000 [00:00<00:00, 27625.31it/s]


finish time: 98.27062797546387


In [162]:
def vectorize_content(data, col="content", new_col="count_vectorized"):
    # Prepare the tf-idf (term frequency-inverse document frequency) TODO: read up on this for report
    start_time = time() 
    count_vectorizer = CountVectorizer(ngram_range=(1, 2))
    tf_idf_transformer = TfidfTransformer(smooth_idf=False)

    # fit and transform train data to count vectorizer
    count_vectorizer.fit(data[col].values)
    count_vect_train = count_vectorizer.transform(data[col].values)
    # fit the counts vector to tfidf transformer
    tf_idf_transformer.fit(count_vect_train)
    count_vect_train = tf_idf_transformer.transform(count_vect_train)
    data[new_col] = [x for x in count_vect_train]
    #cleaned_data_combined['count_vectorized'] = cleaned_data_combined['count_vectorized'].apply(lambda x: tf_idf_transformer.transform([x]))

    end_time = time()
    print(f"Time elapsed of TF IDF transform for {col}:", end_time - start_time)

In [164]:
def split_csr_data(data, features="content", y="type", set="set", get_val=True):
    train = data[data[set] == 0]
    val = data[data[set] == 1]
    test = data[data[set] == 2]
    X_train, y_train = vstack(train[features]), train[y].astype(int)
    X_val, y_val = vstack(val[features]), val[y].astype(int)
    X_test, y_test = vstack(test[features]), test[y].astype(int)
    if not get_val:
        return X_train, X_test, y_train, y_test
    return X_train, X_val, X_test, y_train, y_val, y_test

In [169]:
backup = pd.DataFrame()
def try_models(models, X_train, X_test, y_train, y_test, name=None):
    global backup
    metrics = []
    for model in models:
        start_time = time() 
        model.fit(X_train, y_train)
        train_time = time() - start_time
        y_train_pred = model.predict(X_train)
        y_pred = model.predict(X_test)
        
        if name == None:
            name = type(model).__name__
        metrics.append({
            "name": name,
            "train_acc": accuracy_score(y_train, y_train_pred),
            "test_acc": accuracy_score(y_test, y_pred),
            "precision": precision_score(y_test, y_pred),
            "recall": recall_score(y_test, y_pred),
            "f1": f1_score(y_test, y_pred), 
            "time": "{:.2f}".format(train_time)
        })
        backup = pd.DataFrame(metrics)
        print(f"{name} finished in {(time() - start_time):.2f} seconds")
    return pd.DataFrame(metrics)

In [191]:
class Test_baseline():
    def __init__(self):
        self.metrics = pd.DataFrame()

    def test_baseline(self, X_train, X_test, y_train, y_test, name=None, model=None):
        if model == None:
            model = LogisticRegression()
        metric = try_models([model], X_train, X_test, y_train, y_test, name=name)
        self.metrics = pd.concat([self.metrics, metric])

    def test_cols(self, data, cols_to_test):
        for col, name in cols_to_test:
            self.test_baseline(*split_csr_data(data, features=col, y="type_binary", get_val=False), name=name)

In [ ]:
vectorized_data = pd.read_csv("../datasets/big/dataset_unbalanced_cleaned.csv")
for col in ["content_combined", "content_authors", "content_title", "content_domain", "content_domain_authors_title"]:
    vectorize_content(vectorized_data, col=col, new_col=f"{col}_vectorized")

In [193]:
tests = Test_baseline()

In [ ]:
cols_to_test = [
    ("content_combined_vectorized", "content_combined_unbalanced"),
    ("content_authors_vectorized", "content_authors_unbalanced"), 
    ("content_title_vectorized", "content_title_unbalanced"),
    ("content_domain_vectorized", "content_domain_unbalanced"),
    ("content_domain_authors_title_vectorized", "content_domain_authors_title_unbalanced")]

tests.test_cols(vectorized_data, cols_to_test)
tests.metrics.sort_values(by="f1", ascending=False)

In [ ]:
vectorized_data = pd.read_csv("../datasets/big/dataset_balanced_types_cleaned.csv")
for col in ["content_combined", "content_authors", "content_title", "content_domain", "content_domain_authors_title"]:
    vectorize_content(vectorized_data, col=col, new_col=f"{col}_vectorized")

In [ ]:
cols_to_test = [
    ("content_combined_vectorized", "content_combined_balanced_types"),
    ("content_authors_vectorized", "content_authors_balanced_types"), 
    ("content_title_vectorized", "content_title_balanced_types"),
    ("content_domain_vectorized", "content_domain_balanced_types"),
    ("content_domain_authors_title_vectorized", "content_domain_authors_title_balanced_types")]

tests.test_cols(vectorized_data, cols_to_test)
tests.metrics.sort_values(by="f1", ascending=False)

In [198]:
vectorized_data = pd.read_csv("../datasets/big/dataset_balanced_bin_cleaned.csv")
for col in ["content_combined", "content_authors", "content_title", "content_domain", "content_domain_authors_title"]:
    vectorize_content(vectorized_data, col=col, new_col=f"{col}_vectorized")

Time elapsed of TF IDF transform for content_title: , 20.984931230545044
Time elapsed of TF IDF transform for content_domain: , 18.75245451927185
Time elapsed of TF IDF transform for content_domain_authors_title: , 18.719897985458374


In [199]:
cols_to_test = [
    ("content_combined_vectorized", "content_combined_balanced_bin"),
    ("content_authors_vectorized", "content_authors_balanced_bin"), 
    ("content_title_vectorized", "content_title_balanced_bin"),
    ("content_domain_vectorized", "content_domain_balanced_bin"),
    ("content_domain_authors_title_vectorized", "content_domain_authors_title_balanced_bin")]

tests.test_cols(vectorized_data, cols_to_test)
tests.metrics.sort_values(by="f1", ascending=False)

content_combined_balanced_bin finished in 13.79 seconds
content_authors_balanced_bin finished in 13.79 seconds
content_title_balanced_bin finished in 8.47 seconds
content_domain_balanced_bin finished in 15.09 seconds
content_domain_authors_title_balanced_bin finished in 11.56 seconds


,name,train_acc,test_acc,precision,recall,f1,time
0,content_domain_authors_title_unbalanced,0.978500,0.884,0.898374,0.870079,0.884000,14.94
0,content_domain_authors_title_balanced_bin,0.978500,0.877,0.896970,0.860465,0.878338,11.52
0,content_domain_unbalanced,0.975875,0.869,0.874751,0.866142,0.870425,16.33
0,content_domain_balanced_bin,0.976250,0.866,0.877470,0.860465,0.868885,15.04
0,content_authors_unbalanced,0.974625,0.869,0.887064,0.850394,0.868342,16.88
0,content_authors_balanced_bin,0.973625,0.861,0.888660,0.835271,0.861139,13.74
0,content_title_unbalanced,0.967500,0.839,0.846307,0.834646,0.840436,10.92
0,content_combined_balanced_bin,0.969500,0.838,0.856855,0.823643,0.839921,13.75
0,content_title_balanced_bin,0.968875,0.838,0.859756,0.819767,0.839286,8.43
0,content_combined_unbalanced,0.968250,0.835,0.842315,0.830709,0.836472,15.13
